In [1]:
import datetime as dt

import pandas as pd
import numpy as np
from lenskit.algorithms import item_knn, user_knn
from sklearn.model_selection import ParameterGrid

from recsys4daos.datasets import to_lenskit
from recsys4daos.model_selection import cvtt_open, explore_hparams
from recsys4daos.evaluation import test_with_hparams_lenskit

import paths

/home/daviddavo/recsys4daos/src/recsys4daos/model_selection.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Parameters

In [2]:
# Dataset config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 20  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,15,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_FOLDS = 10 # Use just last 10 splits
LAST_FOLD_DATE_STR: str = None

# Search space config
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
ITEMKNN_Ks = [1,2,3,4,5,6,7,8,9,10,15]

OPTIM_METRIC = 'map@10'

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
ORG_NAME = "Balancer"
SPLITS_FREQ = "3d"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = None


# Load the dataset

In [4]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 602 non-null    object        
 1   author             602 non-null    object        
 2   date               602 non-null    datetime64[us]
 3   start              602 non-null    datetime64[us]
 4   end                602 non-null    datetime64[us]
 5   platform_proposal  602 non-null    object        
dtypes: datetime64[us](3), object(3)
memory usage: 28.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111987 entries, 0 to 111986
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        111987 non-null  object        
 1   proposal  111987 non-null  object        
 2   voter     111987 non-null  object        
 3   date      111987 non-null  datetime64[us]

In [5]:
df = to_lenskit(dfv)
df

,user,item,timestamp,rating
0,0x28143659f4500411d9e49513b88e66cdf81f9002,e41b9c54-4da0-554e-ad10-8a2397d4c2b2,2020-07-18 16:12:04,1
1,0x4aa2c2c321d86e6ce9c5b1fad7c2e084d867dea3,f3a84703-d173-500e-83e5-d5314a5946ef,2020-07-18 16:12:18,1
2,0x911f80d16ec1b70db332227b03beb871ac58ead7,f3a84703-d173-500e-83e5-d5314a5946ef,2020-07-18 16:12:22,1
3,0xd98e44d09660a89022515d6d41662d05a14253e7,f3a84703-d173-500e-83e5-d5314a5946ef,2020-07-18 16:12:39,1
4,0xd98e44d09660a89022515d6d41662d05a14253e7,e41b9c54-4da0-554e-ad10-8a2397d4c2b2,2020-07-18 16:13:31,1
...,...,...,...,...
111982,0xff0281256b2a478905b79895c1a2038a679e1751,4db5212d-7350-5ffd-b666-e43118aa20c0,2023-07-17 16:45:33,1
111983,0x222e3cb8d92bc316e9dc8b45eb218702bf94f9c1,44450f97-8c69-529b-b201-e7c5322ee709,2023-07-17 17:55:48,1
111984,0x222e3cb8d92bc316e9dc8b45eb218702bf94f9c1,3c092794-ba86-54f3-a760-ceb90c1e4c11,2023-07-17 17:56:20,1
111985,0x222e3cb8d92bc316e9dc8b45eb218702bf94f9c1,9fe9a5b6-9833-5801-8fcb-fa5eaa560791,2023-07-17 17:56:48,1


## Split in folds

In [6]:
all_folds = { f.end:f for f in cvtt_open(
    df, SPLITS_FREQ, dfp.reset_index(), remove_not_in_train_col='item', col_item='item', last_fold=LAST_FOLD_DATE_STR,
)}
last_folds_idx = list(all_folds.keys())[-LAST_FOLDS:]
last_folds_idx

[Timestamp('2023-06-18 00:00:00'),
 Timestamp('2023-06-21 00:00:00'),
 Timestamp('2023-06-24 00:00:00'),
 Timestamp('2023-06-27 00:00:00'),
 Timestamp('2023-06-30 00:00:00'),
 Timestamp('2023-07-03 00:00:00'),
 Timestamp('2023-07-06 00:00:00'),
 Timestamp('2023-07-09 00:00:00'),
 Timestamp('2023-07-12 00:00:00'),
 Timestamp('2023-07-15 00:00:00')]

# Item-based KNN

In [7]:
def testHParamsItemKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = item_knn.ItemItem(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsItemKNN(last_folds_idx[-1],5,'14d'))

/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB


found 1 potential runtime problems - see https://boi.st/lkpy-perf


/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/lenskit/algorithms/item_knn.py:119: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  b = blocks[bi]


fold_t             2023-07-15 00:00:00
time_train                    5.467095
time_rec                      0.187589
open_proposals                      13
min_recs                             2
avg_recs                     12.470588
precision@1                   0.588235
precision@3                   0.647059
precision@5                    0.65098
precision@10                  0.596078
precision@15                  0.484967
precision@100                 0.072745
ndcg@1                           0.375
ndcg@3                        0.433379
ndcg@5                        0.456911
ndcg@10                       0.501007
ndcg@15                       0.514801
ndcg@100                      0.514801
map@1                         0.588235
map@3                         0.632353
map@5                         0.655131
map@10                        0.708737
map@15                        0.729471
map@100                       0.729471
recall@1                      0.084994
recall@3                 

## Exploring hparams

In [8]:
results = explore_hparams(
    testHParamsItemKNN, 
    ParameterGrid({
    'fold': last_folds_idx,
    'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
    'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('itemknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfi = pd.DataFrame(results)
mdfi

Restored checkpoint from ../.cache/Balancer/hparams-itemknn_3d_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2023-06-18,1,7d,2023-06-18,0.001602,0.018399,13,1.0,9.384615,0.846154,...,0.926693,1.0,1.0,0.407246,0.588675,0.675570,0.816417,0.863426,0.863426,0.131565
1,2023-06-18,1,14d,2023-06-18,0.001688,0.021941,13,1.0,10.388889,0.722222,...,0.886372,1.0,1.0,0.285421,0.476648,0.564046,0.753647,0.837407,0.837407,0.131466
2,2023-06-18,1,21d,2023-06-18,0.001874,0.024150,13,1.0,10.526316,0.684211,...,0.892353,1.0,1.0,0.285421,0.476648,0.564046,0.753647,0.837407,0.837407,0.131300
3,2023-06-18,1,30d,2023-06-18,0.002826,0.031047,13,1.0,11.041667,0.541667,...,0.914779,1.0,1.0,0.285421,0.476648,0.564046,0.753647,0.837407,0.837407,0.132813
4,2023-06-18,1,60d,2023-06-18,0.007314,0.040116,13,1.0,11.120000,0.560000,...,0.878188,1.0,1.0,0.273962,0.469872,0.567855,0.761231,0.839757,0.839757,0.133674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2023-07-15,15,21d,2023-07-15,0.001590,0.061489,13,2.0,12.557377,0.655738,...,0.896910,1.0,1.0,0.171458,0.398387,0.541411,0.788302,0.889822,0.889822,0.152673
766,2023-07-15,15,30d,2023-07-15,0.002374,0.069769,13,2.0,12.578125,0.609375,...,0.897765,1.0,1.0,0.184237,0.377232,0.542526,0.790845,0.896253,0.896253,0.152889
767,2023-07-15,15,60d,2023-07-15,0.004687,0.089574,13,2.0,12.619718,0.690141,...,0.903485,1.0,1.0,0.230846,0.428850,0.560960,0.773549,0.866012,0.866012,0.154550
768,2023-07-15,15,90d,2023-07-15,0.011711,0.111827,13,2.0,12.625000,0.666667,...,0.904825,1.0,1.0,0.225239,0.428850,0.556574,0.769163,0.861626,0.861626,0.155109


### Best overall hparams

In [9]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfi.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfi[mdfi['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
14d         15    0.002260  6.998182     0.389417      0.268297   
            10    0.002169  6.998182     0.389417      0.268297   
            9     0.002098  6.998182     0.389417      0.268297   
            8     0.002456  6.998182     0.389417      0.268297   
            7     0.002365  6.998182     0.389417      0.268297   
...                    ...       ...          ...           ...   
10YE        5     0.268022  7.115336     0.354501      0.241376   
            1     0.267761  7.115336     0.350946      0.240251   
            4     0.267895  7.115336     0.354322      0.241235   
            3     0.268276  7.115336     0.353095      0.240954   
            2     0.268912  7.115336     0.352736      0.240673   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
14d         15       0.028268  0.335027  0.346991  0.348410  0.469239   
            10       0.028268  0.334747  0.346736  0.348155  0.468823   
            9        0.028268  0.334747  0.346736  0.348155  0.468823   
            8        0.028268  0.334652  0.346660  0.348079  0.468605   
            7        0.028268  0.334305  0.346386  0.347805  0.468169   
...                       ...       ...       ...       ...       ...   
10YE        5        0.025347  0.454180  0.471516  0.473447  0.432871   
            1        0.025347  0.449448  0.469420  0.472501  0.429957   
            4        0.025347  0.449974  0.467597  0.469629  0.428620   
            3        0.025347  0.447929  0.465808  0.468206  0.426120   
            2        0.025347  0.446735  0.464731  0.467447  0.424783   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
14d         15  0.487686  0.489808  0.432388   0.541298    0.555556   
            10  0.487315  0.489438  0.432388   0.541298    0.555556   
            9   0.487315  0.489438  0.432388   0.541298    0.555556   
            8   0.487158  0.489280  0.432388   0.541298    0.555556   
            7   0.486837  0.488960  0.432388   0.541298    0.555556   
...                  ...       ...       ...        ...         ...   
10YE        5   0.449995  0.452008  0.444991   0.544134    0.555556   
            1   0.447715  0.450590  0.434395   0.541457    0.555556   
            4   0.445528  0.447622  0.443573   0.543933    0.555556   
            3   0.443305  0.445646  0.442582   0.542949    0.555556   
            2   0.441464  0.444015  0.440645   0.542175    0.555556   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
14d         15       0.419310        0.507318         0.519542  
            10       0.419310        0.507318         0.519542  
            9        0.419310        0.507318         0.519542  
            8        0.419310        0.507318         0.519542  
            7        0.419310        0.507318         0.519542  
...                       ...             ...              ...  
10YE        5        0.415748        0.496640         0.505790  
            1        0.415092        0.495061         0.504986  
            4        0.416680        0.498611         0.508371  
            3        0.414226        0.495156         0.504916  
            2        0.410982        0.493716         0.503476  

[77 rows x 17 columns]

Now let's see the behaviour in each fold

In [10]:
best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'itemknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Balancer/models/itemknn-best-avg_3d_normalize.parquet


/tmp/ipykernel_3808260/3951593685.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2023-06-18,14d,15,2023-06-18,0.002425,0.021795,13,1.0,10.388889,0.777778,0.648148,0.600000,0.522222,...,0.875261,1.0,1.0,0.282891,0.494505,0.569601,0.753647,0.837407,0.837407,0.130750
2023-06-21,14d,15,2023-06-21,0.006246,0.003724,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067597
2023-06-24,14d,15,2023-06-24,0.002889,0.041867,7,1.0,6.575000,0.850000,0.808333,0.795000,0.520000,...,1.000000,1.0,1.0,0.276050,0.517293,0.764787,0.924561,0.924561,0.924561,0.136960
2023-06-27,14d,15,2023-06-27,0.001526,0.003218,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067916
2023-06-30,14d,15,2023-06-30,0.001523,0.076024,7,2.0,6.934211,0.539474,0.736842,0.752632,0.535526,...,1.000000,1.0,1.0,0.214750,0.434966,0.688197,0.864048,0.864048,0.864048,0.148620
2023-07-03,14d,15,2023-07-03,0.001438,0.014237,7,4.0,6.400000,0.900000,0.833333,0.780000,0.500000,...,1.000000,1.0,1.0,0.285714,0.579365,0.817460,1.000000,1.000000,1.000000,0.126871
2023-07-06,14d,15,2023-07-06,0.001402,0.003210,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068180
2023-07-09,14d,15,2023-07-09,0.002545,0.021964,3,1.0,2.611111,1.000000,0.870370,0.522222,0.261111,...,1.000000,1.0,1.0,0.453704,1.000000,1.000000,1.000000,1.000000,1.000000,0.125873
2023-07-12,14d,15,2023-07-12,0.001343,0.003894,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067322


### Best hparams by fold

This are the parameters used to check which model is the best (validation)

In [11]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfi.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'itemknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Balancer/models/itemknn-best-val_3d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-06-18,1,7d,0.001602,9.384615,0.569231,0.469231,0.055385,0.356665,0.370857,0.378601,0.820769,0.853937,0.879629,0.719757,0.926693,1.0,0.675570,0.816417,0.863426
2023-06-21,15,30d,0.004903,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-06-24,15,7d,0.001378,6.320000,0.792000,0.528000,0.052800,0.398112,0.410978,0.410978,0.914133,0.950146,0.950146,0.791524,1.000000,1.0,0.781524,0.938667,0.938667
2023-06-27,8,10YE,0.263365,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-06-30,15,7d,0.001305,6.913793,0.789655,0.567241,0.056724,0.429109,0.469207,0.469207,0.744315,0.833927,0.833927,0.704680,1.000000,1.0,0.689668,0.872917,0.872917
2023-07-03,6,21d,0.002814,6.400000,0.780000,0.500000,0.050000,0.875539,0.875539,0.875539,0.950000,0.950000,0.950000,0.835714,1.000000,1.0,0.817460,1.000000,1.000000
2023-07-06,10,30d,0.002050,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-07-09,1,14d,0.001376,2.611111,0.522222,0.261111,0.026111,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000
2023-07-12,1,30d,0.004406,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [12]:
best_hparams.tail(len(best_hparams)-1).describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,5.000000,5.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-07-03 00:00:00,0.062876,0.065529,4.111111,2.000000,6.980626,0.450585,0.442201,0.389910,0.265333,...,0.545340,0.555556,0.555556,0.171096,0.337071,0.431374,0.509408,0.518148,0.518148,0.106747
min,2023-06-21 00:00:00,0.001305,0.004701,0.000000,1.000000,2.611111,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067207
25%,2023-06-27 00:00:00,0.001378,0.005282,0.000000,1.000000,6.320000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067589
50%,2023-07-03 00:00:00,0.002814,0.022180,3.000000,2.000000,6.400000,0.551724,0.679325,0.522222,0.261111,...,0.908060,1.000000,1.000000,0.203051,0.426361,0.593717,0.773090,0.851753,0.851753,0.124890
75%,2023-07-09 00:00:00,0.004903,0.054911,7.000000,2.000000,6.913793,0.900000,0.826667,0.780000,0.528000,...,1.000000,1.000000,1.000000,0.286632,0.551048,0.781524,0.938667,0.938667,0.938667,0.133637
max,2023-07-15 00:00:00,0.284290,0.333770,13.000000,4.000000,12.658228,1.000000,0.870370,0.792000,0.567241,...,1.000000,1.000000,1.000000,0.453704,1.000000,1.000000,1.000000,1.000000,1.000000,0.158043
std,NaN,0.119719,0.107204,4.648775,1.224745,3.609219,0.447324,0.422796,0.379915,0.266401,...,0.518171,0.527046,0.527046,0.174634,0.358111,0.423144,0.488016,0.493997,0.493997,0.038302


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb).

In [13]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfi.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'itemknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams[display_columns]

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Balancer/models/itemknn-best-test_3d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-06-21,1,7d,0.001374,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-06-24,15,30d,0.006800,6.638298,0.774468,0.512766,0.051277,0.678619,0.706382,0.706382,0.806791,0.842144,0.842144,0.777406,1.000000,1.0,0.748869,0.926250,0.926250
2023-06-27,15,7d,0.001199,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-06-30,8,10YE,0.266020,6.951456,0.666019,0.466990,0.046699,0.706605,0.768834,0.768834,0.669458,0.741248,0.741248,0.749977,1.000000,1.0,0.678833,0.846951,0.846951
2023-07-03,15,7d,0.001320,5.800000,0.560000,0.340000,0.034000,0.359571,0.359571,0.359571,0.716667,0.716667,0.716667,0.909524,1.000000,1.0,0.682540,0.833333,0.833333
2023-07-06,6,21d,0.001795,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-07-09,10,30d,0.002282,2.740741,0.503704,0.251852,0.025185,0.736111,0.736111,0.736111,0.975309,0.975309,0.975309,1.000000,1.000000,1.0,1.000000,1.000000,1.000000
2023-07-12,1,14d,0.001983,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-07-15,1,30d,0.002259,12.578125,0.621875,0.585938,0.072344,0.555278,0.616434,0.639754,0.644167,0.693597,0.718434,0.484439,0.849265,1.0,0.501900,0.766257,0.878855


In [14]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,5.000000,5.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-07-03 00:00:00,0.031670,0.065333,4.111111,2.000000,6.941724,0.351935,0.394529,0.347341,0.239727,...,0.538807,0.555556,0.555556,0.135595,0.304390,0.401349,0.485866,0.498377,0.498377,0.107791
min,2023-06-21 00:00:00,0.001199,0.002329,0.000000,1.000000,2.740741,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067513
25%,2023-06-27 00:00:00,0.001374,0.003918,0.000000,1.000000,5.800000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068178
50%,2023-07-03 00:00:00,0.001983,0.008261,3.000000,2.000000,6.638298,0.400000,0.614583,0.503704,0.251852,...,0.849265,1.000000,1.000000,0.154762,0.334926,0.501900,0.766257,0.833333,0.833333,0.126136
75%,2023-07-09 00:00:00,0.002282,0.054567,7.000000,2.000000,6.951456,0.593750,0.666667,0.621875,0.466990,...,1.000000,1.000000,1.000000,0.209921,0.476190,0.682540,0.846951,0.878855,0.878855,0.138733
max,2023-07-15 00:00:00,0.266020,0.409199,13.000000,4.000000,12.578125,0.962963,0.839506,0.774468,0.585938,...,1.000000,1.000000,1.000000,0.467949,1.000000,1.000000,1.000000,1.000000,1.000000,0.153047
std,NaN,0.087898,0.131381,4.648775,1.224745,3.564161,0.370276,0.380008,0.337568,0.246510,...,0.513375,0.527046,0.527046,0.157042,0.342302,0.401505,0.465332,0.475221,0.475221,0.038531


# User-based KNN

In [15]:
def testHParamsUserKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = user_knn.UserUser(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsUserKNN(last_folds_idx[-1],5,'14d'))

fold_t             2023-07-15 00:00:00
time_train                    0.593934
time_rec                      0.875164
open_proposals                      13
min_recs                             2
avg_recs                     12.470588
precision@1                   0.764706
precision@3                   0.699346
precision@5                    0.67451
precision@10                       0.6
precision@15                  0.484967
precision@100                 0.072745
ndcg@1                          0.4875
ndcg@3                        0.502518
ndcg@5                        0.520895
ndcg@10                       0.548385
ndcg@15                       0.559436
ndcg@100                      0.559436
map@1                         0.764706
map@3                         0.755447
map@5                         0.765752
map@10                        0.804896
map@15                         0.82206
map@100                        0.82206
recall@1                      0.179626
recall@3                 

## Exploring hparams

In [16]:
results = explore_hparams(
    testHParamsUserKNN, 
    ParameterGrid({
        'fold': last_folds_idx,
        'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
        'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('userknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfu = pd.DataFrame(results)
mdfu

Restored checkpoint from ../.cache/Balancer/hparams-userknn_3d_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2023-06-18,1,7d,2023-06-18,0.000680,0.011365,13,1.0,9.384615,0.615385,...,0.935240,1.0,1.0,0.296074,0.470299,0.582906,0.754701,0.811111,0.811111,0.132788
1,2023-06-18,1,14d,2023-06-18,0.000863,0.014514,13,1.0,10.388889,0.777778,...,0.881434,1.0,1.0,0.336462,0.508761,0.593897,0.753419,0.831944,0.831944,0.131850
2,2023-06-18,1,21d,2023-06-18,0.001056,0.015832,13,1.0,10.526316,0.736842,...,0.887674,1.0,1.0,0.279319,0.419316,0.547379,0.744387,0.828148,0.828148,0.132332
3,2023-06-18,1,30d,2023-06-18,0.001704,0.020097,13,1.0,11.041667,0.666667,...,0.910150,1.0,1.0,0.419404,0.555116,0.642106,0.802422,0.872222,0.872222,0.133149
4,2023-06-18,1,60d,2023-06-18,0.004186,0.024393,13,1.0,11.120000,0.560000,...,0.869077,1.0,1.0,0.273962,0.474664,0.584416,0.785128,0.863333,0.863333,0.133548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2023-07-15,15,21d,2023-07-15,0.000900,0.045391,13,2.0,12.557377,0.672131,...,0.902375,1.0,1.0,0.173374,0.407441,0.532730,0.777886,0.877512,0.877512,0.159555
766,2023-07-15,15,30d,2023-07-15,0.001374,0.048323,13,2.0,12.578125,0.718750,...,0.899005,1.0,1.0,0.188292,0.417730,0.526391,0.770845,0.870450,0.870450,0.161084
767,2023-07-15,15,60d,2023-07-15,0.002732,0.056213,13,2.0,12.619718,0.746479,...,0.904133,1.0,1.0,0.255876,0.468846,0.570314,0.784756,0.874085,0.874085,0.162886
768,2023-07-15,15,90d,2023-07-15,0.005509,0.062086,13,2.0,12.625000,0.708333,...,0.905465,1.0,1.0,0.254473,0.459587,0.567489,0.779106,0.867023,0.867023,0.162568


### Best overall hparams

In [17]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfu.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfu[mdfu['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
14d         4     0.000732  6.998182     0.393023      0.268515   
            10    0.000741  6.998182     0.393143      0.268515   
            9     0.000725  6.998182     0.393143      0.268515   
            8     0.000736  6.998182     0.392587      0.268515   
            7     0.000729  6.998182     0.392587      0.268515   
...                    ...       ...          ...           ...   
10YE        5     0.061577  7.115336     0.351753      0.241235   
            3     0.061464  7.115336     0.350951      0.240954   
            4     0.061780  7.115336     0.350311      0.240813   
            2     0.061466  7.115336     0.346680      0.240813   
            1     0.061798  7.115336     0.343203      0.240110   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
14d         4        0.028268  0.343055  0.352245  0.353484  0.483375   
            10       0.028268  0.342909  0.352032  0.353262  0.483658   
            9        0.028268  0.342909  0.352032  0.353259  0.483658   
            8        0.028268  0.342551  0.351907  0.353137  0.482852   
            7        0.028268  0.342310  0.351666  0.352897  0.482489   
...                       ...       ...       ...       ...       ...   
10YE        5        0.025347  0.453460  0.472979  0.475146  0.431347   
            3        0.025347  0.450160  0.470411  0.473015  0.427665   
            4        0.025347  0.449474  0.469716  0.472332  0.426849   
            2        0.025347  0.444467  0.467042  0.469703  0.420031   
            1        0.025347  0.441141  0.465872  0.470058  0.418410   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
14d         4   0.498548  0.500505  0.442425   0.542048    0.555556   
            10  0.498453  0.500375  0.442593   0.542048    0.555556   
            9   0.498453  0.500360  0.442593   0.542048    0.555556   
            8   0.498145  0.500067  0.442131   0.542048    0.555556   
            7   0.497781  0.499703  0.442131   0.542048    0.555556   
...                  ...       ...       ...        ...         ...   
10YE        5   0.449954  0.452114  0.437352   0.543360    0.555556   
            3   0.446284  0.448696  0.433209   0.542127    0.555556   
            4   0.446275  0.448800  0.435520   0.542255    0.555556   
            2   0.441298  0.443816  0.429907   0.542164    0.555556   
            1   0.441110  0.444130  0.419225   0.536515    0.555556   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
14d         4        0.424390        0.510275         0.520833  
            10       0.425588        0.511117         0.521674  
            9        0.425588        0.511117         0.521674  
            8        0.425101        0.511117         0.521674  
            7        0.425101        0.511117         0.521674  
...                       ...             ...              ...  
10YE        5        0.413478        0.494426         0.504351  
            3        0.410806        0.490938         0.500698  
            4        0.411245        0.491872         0.501797  
            2        0.406834        0.488296         0.498221  
            1        0.408030        0.489635         0.499560  

[77 rows x 17 columns]

In [18]:
best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'userknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Balancer/models/userknn-best-avg_3d_normalize.parquet


/tmp/ipykernel_3808260/3337470013.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2023-06-18,14d,4,2023-06-18,0.000863,0.014466,13,1.0,10.388889,0.833333,0.722222,0.633333,0.522222,...,0.875261,1.0,1.0,0.330698,0.505088,0.609522,0.769044,0.847569,0.847569,0.130966
2023-06-21,14d,4,2023-06-21,0.000818,0.002696,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066323
2023-06-24,14d,4,2023-06-24,0.000774,0.028249,7,1.0,6.575000,0.925000,0.816667,0.800000,0.520000,...,1.000000,1.0,1.0,0.289704,0.539223,0.775752,0.935526,0.935526,0.935526,0.136438
2023-06-27,14d,4,2023-06-27,0.000801,0.002295,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065987
2023-06-30,14d,4,2023-06-30,0.000780,0.051344,7,2.0,6.934211,0.881579,0.736842,0.752632,0.535526,...,1.000000,1.0,1.0,0.236141,0.443717,0.689914,0.860880,0.860880,0.860880,0.147604
2023-07-03,14d,4,2023-07-03,0.000734,0.008760,7,4.0,6.400000,0.800000,0.800000,0.780000,0.500000,...,1.000000,1.0,1.0,0.258929,0.526786,0.794643,1.000000,1.000000,1.000000,0.125600
2023-07-06,14d,4,2023-07-06,0.000687,0.002312,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065182
2023-07-09,14d,4,2023-07-09,0.000692,0.014479,3,1.0,2.611111,1.000000,0.870370,0.522222,0.261111,...,1.000000,1.0,1.0,0.453704,1.000000,1.000000,1.000000,1.000000,1.000000,0.125758
2023-07-12,14d,4,2023-07-12,0.000634,0.002813,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066153


### Best hparams by fold

In [19]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfu.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'userknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Balancer/models/userknn-best-val_3d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-06-18,8,7d,0.000607,9.384615,0.553846,0.469231,0.055385,0.354923,0.373776,0.381603,0.808718,0.854367,0.880636,0.700526,0.926693,1.0,0.654736,0.795584,0.842593
2023-06-21,2,90d,0.008838,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-06-24,8,7d,0.000565,6.320000,0.800000,0.528000,0.052800,0.395930,0.406944,0.406944,0.911467,0.941257,0.941257,0.798190,1.000000,1.0,0.774857,0.932000,0.932000
2023-06-27,4,30d,0.001291,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-06-30,5,7d,0.000535,6.913793,0.789655,0.567241,0.056724,0.463149,0.500396,0.500396,0.808396,0.889865,0.889865,0.704680,1.000000,1.0,0.686341,0.866374,0.866374
2023-07-03,10,10YE,0.061327,6.454545,0.727273,0.463636,0.046364,0.927416,0.927416,0.927416,0.901515,0.901515,0.901515,0.850649,1.000000,1.0,0.817460,1.000000,1.000000
2023-07-06,7,30d,0.001244,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2023-07-09,5,7d,0.000444,2.363636,0.472727,0.236364,0.023636,0.305556,0.305556,0.305556,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000
2023-07-12,7,21d,0.000856,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [20]:
best_hparams.tail(len(best_hparams)-1).describe()[display_columns]

,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
count,9.000000,5.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.008416,6.904513,0.385343,0.266138,0.028030,0.290996,0.299227,0.300457,0.488330,0.504897,0.506819,0.439541,0.542048,0.555556,0.427139,0.511336,0.521893
min,0.000444,2.363636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000565,6.320000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000856,6.454545,0.472727,0.236364,0.023636,0.305556,0.305556,0.305556,0.773595,0.811440,0.828737,0.602345,0.878436,1.000000,0.565597,0.803649,0.866374
75%,0.001291,6.913793,0.727273,0.528000,0.052800,0.463149,0.500396,0.500396,0.901515,0.901515,0.901515,0.798190,1.000000,1.000000,0.774857,0.932000,0.932000
max,0.061327,12.470588,0.800000,0.600000,0.072745,0.927416,0.927416,0.927416,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,0.020021,3.610574,0.377462,0.272468,0.029442,0.324087,0.329610,0.330694,0.467608,0.481495,0.482904,0.430311,0.515667,0.527046,0.420948,0.488863,0.496925


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb)

In [21]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfu.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'userknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Balancer/models/userknn-best-test_3d_normalize.parquet


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,k,window_size,,,,,,,,,,,,,,,,,,,,,
2023-06-21,8,7d,2023-06-21,0.000586,0.001775,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065672
2023-06-24,2,90d,2023-06-24,0.008496,0.049895,7,1.0,6.690909,0.781818,0.745455,0.745455,0.489091,...,1.000000,1.0,1.0,0.234275,0.500828,0.754037,0.921739,0.921739,0.921739,0.138099
2023-06-27,8,7d,2023-06-27,0.000459,0.001636,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065626
2023-06-30,4,30d,2023-06-30,0.001314,0.057033,7,2.0,6.939024,0.878049,0.735772,0.746341,0.530488,...,1.000000,1.0,1.0,0.242361,0.452381,0.696320,0.867316,0.867316,0.867316,0.149253
2023-07-03,5,7d,2023-07-03,0.000517,0.005104,7,4.0,5.800000,0.600000,0.600000,0.560000,0.340000,...,1.000000,1.0,1.0,0.436508,0.607143,0.761905,0.875000,0.875000,0.875000,0.124545
2023-07-06,10,10YE,2023-07-06,0.060979,0.118111,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066396
2023-07-09,7,30d,2023-07-09,0.001414,0.021943,3,1.0,2.740741,0.925926,0.839506,0.503704,0.251852,...,1.000000,1.0,1.0,0.446667,1.000000,1.000000,1.000000,1.000000,1.000000,0.129916
2023-07-12,5,7d,2023-07-12,0.000420,0.002068,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066061
2023-07-15,7,21d,2023-07-15,0.000877,0.043789,13,2.0,12.557377,0.688525,0.683060,0.655738,0.591803,...,0.890782,1.0,1.0,0.188521,0.400431,0.526897,0.761056,0.858515,0.858515,0.154468


In [22]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,5.000000,5.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-07-03 00:00:00,0.008340,0.033484,4.111111,2.000000,6.945610,0.430480,0.400421,0.356804,0.244804,...,0.543420,0.555556,0.555556,0.172037,0.328976,0.415462,0.491679,0.502508,0.502508,0.106671
min,2023-06-21 00:00:00,0.000420,0.001636,0.000000,1.000000,2.740741,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065626
25%,2023-06-27 00:00:00,0.000517,0.002068,0.000000,1.000000,5.800000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066061
50%,2023-07-03 00:00:00,0.000877,0.021943,3.000000,2.000000,6.690909,0.600000,0.600000,0.503704,0.251852,...,0.890782,1.000000,1.000000,0.188521,0.400431,0.526897,0.761056,0.858515,0.858515,0.124545
75%,2023-07-09 00:00:00,0.001414,0.049895,7.000000,2.000000,6.939024,0.781818,0.735772,0.655738,0.489091,...,1.000000,1.000000,1.000000,0.242361,0.500828,0.754037,0.875000,0.875000,0.875000,0.138099
max,2023-07-15 00:00:00,0.060979,0.118111,13.000000,4.000000,12.557377,0.925926,0.839506,0.746341,0.591803,...,1.000000,1.000000,1.000000,0.446667,1.000000,1.000000,1.000000,1.000000,1.000000,0.154468
std,NaN,0.019906,0.038790,4.648775,1.224745,3.554933,0.419196,0.384930,0.347177,0.252618,...,0.516689,0.527046,0.527046,0.184677,0.355390,0.412019,0.470497,0.478525,0.478525,0.039658
